In [35]:
import time
import numpy as np
import pandas as pd

class EM_Geometric(object):
    def __init__(self, num_clusters, min_step=1e-4, probs=None, priors=None):
        '''
        Initialize the EM algorithm based on given parameters

        Parameters
        ----------
        num_clusters : int
            The number of labels the EM algorithm should form clusters for
        min_step : int
            The minimum steps required for termination of the algorithm
        probs : np.array
            The initial parameter p for each label (geometric distribution)
        priors : np.array
            The initial mixing proportion of each label (aka theta)
        '''
        self.num_clusters = num_clusters
        self.min_step = min_step

        if probs is None:
            self.probs = np.random.random(self.num_clusters)
        else:
            self.probs = probs

        if priors is None:
            self.pis = np.ones(self.num_clusters) / self.num_clusters
        else:
            self.pis = priors

    def cluster(self, data, max_iters=50):
        '''
        Train on the given data to form clusters for the labels

        Parameters
        ----------
        data : pd.DataFrame
            The feature values of the data set
        '''
        self.data = data
        start_time = time.time()

        # Initialize the soft cluster assignments q for each data point to each label
        self.q = np.asmatrix(np.empty((len(self.data), self.num_clusters), dtype=float))

        num_iters = 0

        # Current and previous log-likelihoods
        curr_ll = 0
        prev_ll = 0

        # While termination conditions have not been met, run the algorithm
        while num_iters <= 1 or (curr_ll - prev_ll > self.min_step and num_iters < max_iters):
            # Perform the expectation step
            self.e_step()
            # Perform the maximization step
            self.m_step()

            num_iters += 1

            # Calculate and compare log-likelihoods
            prev_ll = curr_ll
            curr_ll = self.loglikelihood()
            print(f'Iteration {num_iters}, log-likelihood: {curr_ll}')

        total_time = time.time() - start_time
        print(f'Complete, time elapsed: {total_time} seconds')

        # Sort the clusters and re-arrange the soft labels
        i = np.argsort(self.probs)
        self.probs = np.sort(self.probs)
        self.q = self.q[:, i]

    def loglikelihood(self):
        '''
        Calculate the current log-likelihood given data and soft assignments

        Returns
        -------
        ll : float
            The log-likelihood of current set of labels (according to a geometric distribution)
        '''
        # TODO: Implement the log-likelihood function
        N=len(self.data)
        K=self.num_clusters
        L=0
        for n in range(N):
            for k in range(K):
                L+=self.q[n,k]*np.log((1-self.probs[k])**(self.data[n]-1)*self.probs[k]*self.pis[k])

        return L

    def e_step(self):
        '''
        Run the expectation step, calculating the soft assignments based on current parameter estimates
        '''
        N=len(self.data)
        K=self.num_clusters
        for n in range(N):
            for k in range(K):
                self.q[n,k]=self.pis[k]*(1-self.probs[k])**(self.data[n]-1)*self.probs[k]
        for n in range(N):
            #sumq=np.sum(self.q[n,:])
            #print("qn,sumq and its shape",self.q[n,:],sumq,sumq.shape)
            self.q[n,:]=self.q[n,:]/np.sum(self.q[n,:])
            
        
                
       

    def m_step(self):
        '''
        Run the maximization step, calculating parameter estimates based on current soft assignments
        '''
        N=len(self.data)
        K=self.num_clusters
        
        
        for k in range(K):
            thetaktemp=0
            probtemp=0
            
            for n in range(N):
                thetaktemp+= self.q[n,k]
                probtemp+=self.q[n,k]*self.data[n]
                
            self.pis[k]=thetaktemp/N
            self.probs[k]=thetaktemp/probtemp
            
        
                
        

    def get_labels(self):
        '''
        Return the label with highest probability among the soft assignments for each data point
        '''
        return np.array([np.argmax(q) for q in np.array(self.q)])

def generate_geom_data(num_data, cluster_probs):
    '''
    Generate an equal number of data points for each cluster, based on geometric distribution
    '''
    data = np.array([])
    labels = np.array([], dtype=int)
    for i, prob in enumerate(cluster_probs):
        data = np.concatenate((data, np.random.geometric(p=prob, size=num_data)))
        labels = np.concatenate((labels, np.full(num_data, i, dtype=int)))
    return data, labels

def main():
    # TODO: Toggle these between 10 / 1000 and [0.1, 0.5, 0.9] / [0.1, 0.2, 0.9]
    num_data = 1000
    cluster_probs = [0.1, 0.5, 0.9]

    # Do not edit the below code, it is to help you run the algorithm
    # -------------------------------------------------------------------------------
    data, labels = generate_geom_data(num_data=num_data, cluster_probs=cluster_probs)

    em_algo = EM_Geometric(num_clusters=3)
    em_algo.cluster(data)
    print(f'Final probabilities: {em_algo.probs}')

    accuracy = np.equal(em_algo.get_labels(), labels).sum() / len(labels)
    print(f'Accuracy: {accuracy}')

if __name__ == "__main__":
    main()

Iteration 1, log-likelihood: -8571.69294006867
Iteration 2, log-likelihood: -8320.02884556884
Iteration 3, log-likelihood: -8208.6578063601
Iteration 4, log-likelihood: -8152.608140162047
Iteration 5, log-likelihood: -8120.830959842039
Iteration 6, log-likelihood: -8101.678000417236
Iteration 7, log-likelihood: -8089.868785585112
Iteration 8, log-likelihood: -8082.572300991795
Iteration 9, log-likelihood: -8078.126467313038
Iteration 10, log-likelihood: -8075.514465502852
Iteration 11, log-likelihood: -8074.100201142337
Iteration 12, log-likelihood: -8073.478625293053
Iteration 13, log-likelihood: -8073.387313459189
Iteration 14, log-likelihood: -8073.653347494426
Complete, time elapsed: 1.7509608268737793 seconds
Final probabilities: [0.09341985 0.36455863 0.84168845]
Accuracy: 0.656
